In [20]:
# Imports
from __future__ import absolute_import, division, print_function
import string
import os
import pickle
from six.moves import urllib

import tflearn
from tflearn.data_utils import *
import unicodedata

In [21]:
# Get text data from the three Lord of the Rings books
path = 'input.txt'
if not os.path.isfile(path):
  urllib.request.urlretrieve("https://gist.githubusercontent.com/Blabby/eb5064cdc9f6b55f10a06bb2aabd2b04/raw/780dc55ed24122c615557981441c8a811905d4de/LOR_input_c.txt", path)

# Get pickle file containing the dictionary that maps characters to numbers
char_idx_file = 'char_idx_LOR.pickle'
char_idx = None

if os.path.isfile(char_idx_file):
  print('Loading previous char_idx')
  char_idx = pickle.load(open(char_idx_file, 'rb'))

Loading previous char_idx


In [22]:
# Create char_idx and X, Y data
maxlen = 25
X, Y, char_idx = textfile_to_semi_redundant_sequences(path, seq_maxlen=maxlen, redun_step=3)

pickle.dump(char_idx, open(char_idx_file,'wb'))

Vectorizing text...
Text total length: 2,528,442
Distinct chars   : 43
Total sequences  : 842,806


In [23]:
print(char_idx)

{'\x80': 38, '\n': 0, '\x99': 40, '\x98': 39, '\x9c': 41, '!': 2, ' ': 1, '"': 3, "'": 4, ')': 6, '(': 5, '-': 8, ',': 7, '.': 9, ';': 10, '?': 11, '\xe2': 42, 'a': 12, 'c': 14, 'b': 13, 'e': 16, 'd': 15, 'g': 18, 'f': 17, 'i': 20, 'h': 19, 'k': 22, 'j': 21, 'm': 24, 'l': 23, 'o': 26, 'n': 25, 'q': 28, 'p': 27, 's': 30, 'r': 29, 'u': 32, 't': 31, 'w': 34, 'v': 33, 'y': 36, 'x': 35, 'z': 37}


In [7]:
# Create network layers
g = tflearn.input_data([None, maxlen, len(char_idx)])
g = tflearn.lstm(g, 512, return_seq=True)
g = tflearn.dropout(g, 0.5)
g = tflearn.lstm(g, 512, return_seq=True)
g = tflearn.dropout(g, 0.5)
g = tflearn.lstm(g, 512)
g = tflearn.dropout(g, 0.5)
g = tflearn.fully_connected(g, len(char_idx), activation='softmax')
g = tflearn.regression(g, optimizer='adam', loss='categorical_crossentropy',
                       learning_rate=0.001)

In [8]:
m = tflearn.SequenceGenerator(g, dictionary=char_idx,
                              seq_maxlen=maxlen,
                              clip_gradients=10.0,
                              checkpoint_path='model_LOR',
                              tensorboard_verbose=2)

In [ ]:
# Train the neural network
m.fit(X, Y, validation_set=0.15, batch_size=128, n_epoch=10, snapshot_step=2000, run_id='LOR_test')

# Save the trained neural network
m.save('LOR_network.tflearn')

Training Step: 4530  | total loss: 1.77558 | time: 1.860s
| Adam | epoch: 001 | loss: 1.77558 -- iter: 579840/716385


In [16]:
# --- Export from Google Data ---
# http://stackoverflow.com/questions/35719627/how-do-i-quickly-get-data-out-of-a-google-cloud-datalab-notebook

from datalab.context import Context
import datalab.storage as storage

sample_bucket_name = Context.default().project_id + '-datalab-example'
sample_bucket_path = 'gs://' + sample_bucket_name

sample_bucket = storage.Bucket(sample_bucket_name)

# Create storage bucket if it does not exist
if not sample_bucket.exists():
    sample_bucket.create()

# Write an item to the storage bucket
# sample_item = sample_bucket.item('stringtofile.txt')
# sample_item.write_to('This is a string', 'text/plain')

In [19]:
%%bash
gsutil cp 'LOR_network.tflearn' gs://neural-152015-datalab-example/

Copying file://LOR_prediction.tflearn [Content-Type=application/octet-stream]...
|
Operation completed over 1 objects/61.3 MiB.                                     
